<a href="https://colab.research.google.com/github/LuckyBoy587/SchoolaNova/blob/main/BERT_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sentence_transformers import SentenceTransformer, util

# 1. Load embedding model (pretrained on similarity tasks)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


# 2. Sample NCERT-like paragraphs
docs = [
    "Photosynthesis is the process by which green plants make food using sunlight, carbon dioxide, and water.",
    "Respiration is the process by which living organisms release energy from food molecules.",
    "Chlorophyll is a green pigment in plants that helps capture sunlight for photosynthesis."
]

# 3. Encode the paragraphs
doc_embeddings = model.encode(docs, convert_to_tensor=True)

# 4. Encode a sample question
question = "How do plants cook themselves?"
query_embedding = model.encode(question, convert_to_tensor=True)

# 5. Compute similarity
cosine_scores = util.cos_sim(query_embedding, doc_embeddings)
print(cosine_scores)

# 6. Retrieve best paragraph
best_idx = cosine_scores.argmax()
print("Question:", question)
print("Best Match:", docs[best_idx])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tensor([[0.5394, 0.3054, 0.3463]])
Question: How do plants cook themselves?
Best Match: Photosynthesis is the process by which green plants make food using sunlight, carbon dioxide, and water.


In [ ]:
def split_into_chunks(text):
    return text.splitlines()

In [ ]:
file_chunks = []
with open("/content/drive/MyDrive/ColabContent/photosynthesis.txt") as f:
  file_chunks = split_into_chunks(f.read())
  print(*file_chunks, sep='\n')

Photosynthesis definition states that the process exclusively takes place in the chloroplasts through photosynthetic pigments such as chlorophyll a, chlorophyll b, carotene and xanthophyll. All green plants and a few other autotrophic organisms utilize photosynthesis to synthesize nutrients by using carbon dioxide, water and sunlight. The by-product of the photosynthesis process is oxygen.Let us have a detailed look at the process, reaction and importance of photosynthesis.
What Is Photosynthesis in Biology?
The word “photosynthesis” is derived from the Greek words phōs (pronounced: “fos”) and σύνθεσις (pronounced: “synthesis“)Phōs means “light” and σύνθεσις means, “combining together.” This means “combining together with the help of light.”
Photosynthesis also applies to other organisms besides green plants. These include several prokaryotes such as cyanobacteria, purple bacteria and green sulfur bacteria. These organisms exhibit photosynthesis just like green plants.The glucose produ

In [ ]:
query = "Where did the name photosysnthesis came from?"

In [ ]:
chunk_embeddings = model.encode(file_chunks, convert_to_tensor=True)
query_embedding = model.encode(query, convert_to_tensor=True)
cosine_scores = util.cos_sim(query_embedding, chunk_embeddings)
best_idx = cosine_scores.argmax()

print("Question:", query)
print("Best Match:", file_chunks[best_idx])

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.1 MB/s eta 0:00:00


In [29]:
import pdfplumber, re
from nltk.tokenize import sent_tokenize

def clean_text(text: str) -> str:
    """Aggressively clean NCERT/School-book style PDF text with OCR noise."""
    # Normalize spaces/newlines
    text = re.sub(r'\s+', ' ', text)

    # Remove OCR garbled "CChhaapptteerr" like strings
    text = re.sub(r'C+H*A+P+T+E+R+.*?\d+', '', text, flags=re.IGNORECASE)

    # Remove lines with .indd and timestamps (OCR file tags)
    text = re.sub(r'\.?i+n+d+d+\s*\d+.*?(AM|PM)?', '', text, flags=re.IGNORECASE)

    # Remove publisher footer lines
    text = re.sub(r'Curiosity.*?(Grade|Gr\.a\.d\.e)', '', text, flags=re.IGNORECASE)

    # Remove "Chapter ..." repeated headers (even broken ones)
    text = re.sub(r'Chapter\s+The.*?Solutions', '', text, flags=re.IGNORECASE)

    # Remove random numbers/dates like //2288//22002255 ::0066::3366
    text = re.sub(r'[/:\d]+', '', text)

    # Collapse multiple punctuation (.... → . , ??? → ? , !!! → !)
    text = re.sub(r'([.?!])\1+', r'\1', text)

    # Remove bullets/list markers
    text = re.sub(r'[\\•\·\●\-\–\—\»\"z]', '', text)

    # Normalize ligatures
    text = text.replace('ﬁ', 'fi').replace('ﬂ', 'fl')

    return text.strip()


def pdf_to_sentences(pdf_path: str):
    """Extract sentences from PDF and clean them."""
    all_sentences = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            raw_text = page.extract_text()
            if not raw_text:
                continue
            # Clean and split into sentences
            cleaned = clean_text(raw_text)
            sentences = sent_tokenize(cleaned)
            for s in sentences:
                s = s.strip()
                if len(s.split()) > 5:  # skip very tiny fragments
                    all_sentences.append(s)
    return all_sentences

In [30]:
print(*pdf_to_sentences("/content/drive/MyDrive/ColabContent/ch9.pdf"), sep='\n')

The Amaing World of Solutes, Solvents, and Solutions The Amaing World  of Solutes, Solvents, and Solutions Probe and ponder  What do you think is happening in the picture above?
What happens when you add too much sugar to your tea and it stops dissolving?
How can you solve this problem?
Why do sugar and salt dissolve in water but not in oil?
Why is water considered a good solvent?
Why are water bottles usually tall and cylindrical in shape instead of spherical?
The Amaing World of Solutes, Solvents, and Solutions  You must have taken an Oral Rehydration Solution (ORS) at some time in your life.
ORS is used to treat dehydration by keeping your body hydrated.
You have learnt to prepare ORS at home in  .
You may have wondered why every sip of your homemade ORS tastes the same, no matter how much you drink.
Why does it not taste salty in one sip and sweet in another?
This is because when you add sugar and salt to water, they form a mixture in which the components are evenly distributed Fig

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [33]:
ch9_sentences = pdf_to_sentences("/content/drive/MyDrive/ColabContent/ch9.pdf")

In [42]:
import faiss
import numpy as np

ch9_embeddings = model.encode(ch9_sentences, convert_to_tensor=False) # Convert to numpy array for FAISS

# create a FAISS index
d = ch9_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(ch9_embeddings)

print(f"Created FAISS index with {index.ntotal} vectors.")

Created FAISS index with 323 vectors.


In [43]:
my_question = "What is a solute and a solvent?"
my_question_embd = model.encode(my_question, convert_to_tensor=True)

In [44]:
import torch

# Encode the question and convert to numpy
my_question_embd_np = model.encode([my_question], convert_to_tensor=False)

# Search the FAISS index
top_k = 5
distances, indices = index.search(my_question_embd_np, top_k)

print("Question:", my_question)
print(f"Top {top_k} Matches:")
for i in range(top_k):
    print(f"- {ch9_sentences[indices[0][i]]}")

Question: What is a solute and a solvent?
Top 5 Matches:
- What Are Solute, Solvent, and Solution?
- In a solution formed by mixing two liquids, the component present in less quantity is known as solute and the other component is called solvent.
- Solvent Solute + Solvent Solution When a solution is formed by mixing two liquids, it is not always clear which substance is dissolving the other.
- Magnified schematic picture of a cases, the substance present in smaller solute evenly distributed in a solvent amount is called the solute, while the one in larger amount is called the solvent.
- Whenever a solid is mixed with a liquid to form a solution, the solid component is called the solute, and the liquid component is called the solvent.


In [41]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 23.0 MB/s eta 0:00:00
